In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
pip install gplearn scikit-learn


In [ ]:
# Paths to the CSV files
train_features_path = '/content/drive/MyDrive/ngsim 2/ngsim_v_dt=1/train_feature.csv'
train_labels_path = '/content/drive/MyDrive/ngsim 2/ngsim_v_dt=1/train_label.csv'
validation_features_path = '/content/drive/MyDrive/ngsim 2/ngsim_v_dt=1/validation_feature.csv'
validation_labels_path = '/content/drive/MyDrive/ngsim 2/ngsim_v_dt=1/validation_label.csv'

In [ ]:
import pandas as pd

In [ ]:
# Loading the CSV files without headers, since we will set column names manually
train_features_df = pd.read_csv(train_features_path, header=None)
train_labels_df = pd.read_csv(train_labels_path, header=None)
validation_features_df = pd.read_csv(validation_features_path, header=None)
validation_labels_df = pd.read_csv(validation_labels_path, header=None)

In [ ]:
# Set column names
train_features_df.columns = ['dx', 'dv', 'v']
train_labels_df.columns = ['v_next']
validation_features_df.columns = ['dx', 'dv', 'v']
validation_labels_df.columns = ['v_next']


In [ ]:
# Concatenate features and labels
train_df = pd.concat([train_features_df, train_labels_df], axis=1)
validation_df = pd.concat([validation_features_df, validation_labels_df], axis=1)

In [ ]:
# Split into features and labels
X_train = train_df[['dx', 'dv', 'v']].values
y_train = train_df['v_next'].values
X_validation = validation_df[['dx', 'dv', 'v']].values
y_validation = validation_df['v_next'].values


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import os
import numpy as np

In [ ]:
# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_validation = torch.tensor(X_validation, dtype=torch.float32)
y_validation = torch.tensor(y_validation, dtype=torch.float32).view(-1, 1)


In [ ]:
# DataLoader for batch processing
batch_size = 64
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_dataset = TensorDataset(X_validation, y_validation)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch
import torch.nn as nn

class SymbolicLayer(nn.Module):
    """Simple symbolic layer applying a predefined function to a sliced input."""
    def __init__(self, function, slice_index):
        super().__init__()
        self.function = function
        self.slice_index = slice_index  # Define which part of input this layer should take

    def forward(self, x):
        # Apply function to a specific part of the input (e.g., one feature)
        return self.function(x[:, self.slice_index:self.slice_index+1])

class Square(nn.Module):
    def forward(self, x):
        return x ** 2

class Identity(nn.Module):
    def forward(self, x):
        return x

class SymbolicNet(nn.Module):
    """A simple symbolic regression network that ensures a fixed number of outputs."""
    def __init__(self, funcs, input_features):
        super().__init__()
        self.hidden_layers = nn.ModuleList()
        # Create layers and assign each to process a specific input feature
        assert len(funcs) == 5  # Ensure there are exactly 5 functions
        for i in range(5):
            # Assign each function to a specific input feature
            # Assume input_features is at least as large as len(funcs)
            self.hidden_layers.append(SymbolicLayer(funcs[i % len(funcs)], i % input_features))

        self.output_weight = nn.Parameter(torch.randn(5, 1))  # This matches the 5 symbolic layers

    def forward(self, x):
        outputs = []
        for layer in self.hidden_layers:
            outputs.append(layer(x))
        x = torch.cat(outputs, dim=1)  # Concatenate all layer outputs
        x = torch.matmul(x, self.output_weight)  # Multiply by the output weight
        return x

# Initialize the network with 5 instances of functions
funcs = [Square(), Identity(), Square(), Identity(), Square()]  # Adjusted list to have exactly 5 elements
net = SymbolicNet(funcs=funcs, input_features=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

# Assume train_loader and validation_loader are defined correctly
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    net.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    net.eval()
    validation_loss = 0
    with torch.no_grad():
        for inputs, targets in validation_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            validation_loss += criterion(outputs, targets).item()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {validation_loss / len(validation_loader)}')


Epoch 1, Loss: 3990.19384765625, Validation Loss: 13197.075858200653
Epoch 2, Loss: 21285.33203125, Validation Loss: 5147.203819564626
Epoch 3, Loss: 1891.1995849609375, Validation Loss: 2810.648464540892
Epoch 4, Loss: 1795.120361328125, Validation Loss: 1600.586901314651
Epoch 5, Loss: 1502.567626953125, Validation Loss: 862.8171315253535
Epoch 6, Loss: 246.92587280273438, Validation Loss: 435.2807819028444
Epoch 7, Loss: 128.31495666503906, Validation Loss: 220.5752923217001
Epoch 8, Loss: 68.9952621459961, Validation Loss: 125.7171401977539
Epoch 9, Loss: 114.23883819580078, Validation Loss: 89.70983483519736
Epoch 10, Loss: 98.37596130371094, Validation Loss: 77.21655041658425
Epoch 11, Loss: 104.00625610351562, Validation Loss: 72.09335853480086
Epoch 12, Loss: 64.01806640625, Validation Loss: 68.31754923470413
Epoch 13, Loss: 43.97951126098633, Validation Loss: 63.95114565499221
Epoch 14, Loss: 45.623294830322266, Validation Loss: 59.56732675093639
Epoch 15, Loss: 36.84711837768

In [ ]:
# Retrieve output weights from the trained model
output_weights = net.output_weight.detach().cpu().numpy().flatten()  # Flatten to make indexing easier

# Define the function for each symbolic transformation
# Assuming funcs are applied cyclically to input features dx, dv, v
input_features = ['dx', 'dv', 'v']
symbolic_equations = []

for i, layer in enumerate(net.hidden_layers):
    func_type = type(layer.function).__name__
    feature = input_features[i % len(input_features)]

    # Depending on the function type, format the symbolic expression
    if func_type == 'Square':
        expr = f"({feature}^2)"
    elif func_type == 'Identity':
        expr = f"({feature})"
    else:
        expr = f"({feature})"  # Default case, add more cases if you have more function types

    # Multiply by the corresponding weight and add to the list
    weight = output_weights[i]
    symbolic_equations.append(f"{weight:.3f} * {expr}")

# Combine all parts into a single equation string
final_equation = " + ".join(symbolic_equations) + " + b"
print("Symbolic Equation:", final_equation)


Symbolic Equation: -0.007 * (dx^2) + 0.222 * (dv) + 0.056 * (v^2) + 0.390 * (dx) + -0.030 * (dv^2) + b


In [ ]:
import torch
import torch.nn as nn

class Cubic(nn.Module):
    def forward(self, x):
        return x ** 3

class Sine(nn.Module):
    def forward(self, x):
        return torch.sin(x)

class Cosine(nn.Module):
    def forward(self, x):
        return torch.cos(x)

class Exponential(nn.Module):
    def forward(self, x):
        return torch.exp(x)

class Logarithm(nn.Module):
    def forward(self, x):
        # To avoid issues with log(0) and negative numbers, apply a small shift; adjust as needed based on your data range
        return torch.log(x + 1)

class SymbolicNet(nn.Module):
    """A simple symbolic regression network with diverse symbolic functions."""
    def __init__(self, funcs, input_features):
        super().__init__()
        self.hidden_layers = nn.ModuleList()
        # Assume that len(funcs) will correctly distribute across the number of outputs
        for i in range(len(funcs)):
            # Assign each function to a specific input feature, cycling through input features
            self.hidden_layers.append(SymbolicLayer(funcs[i], i % input_features))

        # Adjust the output weights to match the number of functions
        self.output_weight = nn.Parameter(torch.randn(len(funcs), 1))

    def forward(self, x):
        outputs = []
        for layer in self.hidden_layers:
            outputs.append(layer(x))
        x = torch.cat(outputs, dim=1)
        x = torch.matmul(x, self.output_weight)
        return x

# Define a richer set of functions
funcs = [
    Square(), Identity(), Cubic(), Sine(), Cosine(),
    Exponential(), Logarithm(), Square(), Identity(), Square()
]  # Extended list with more diverse functions

net = SymbolicNet(funcs=funcs, input_features=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

# Assume train_loader and validation_loader are defined correctly
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    net.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    net.eval()
    validation_loss = 0
    with torch.no_grad():
        for inputs, targets in validation_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            validation_loss += criterion(outputs, targets).item()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {validation_loss / len(validation_loader)}')


Epoch 1, Loss: 59156596.0, Validation Loss: 2769026488.464794
Epoch 2, Loss: 2415624.75, Validation Loss: 2842422.2492088606
Epoch 3, Loss: 2739858.25, Validation Loss: 2691829.9485759493
Epoch 4, Loss: 1067607.375, Validation Loss: 2492839.128164557
Epoch 5, Loss: 4419076.0, Validation Loss: 2298383.6004746836
Epoch 6, Loss: 1176516.125, Validation Loss: 2057017.9335443038
Epoch 7, Loss: 1218706.375, Validation Loss: 1818321.05221519
Epoch 8, Loss: 1173187.25, Validation Loss: 1556470.7954905063
Epoch 9, Loss: 849842.8125, Validation Loss: 1348398.7448575948
Epoch 10, Loss: 1651477.25, Validation Loss: 1080821.4909018988
Epoch 11, Loss: 522408.5625, Validation Loss: 885257.3433544304
Epoch 12, Loss: 476060.78125, Validation Loss: 718090.163568038
Epoch 13, Loss: 636573.3125, Validation Loss: 585781.2480221519
Epoch 14, Loss: 339830.625, Validation Loss: 437208.51720727846
Epoch 15, Loss: 285594.84375, Validation Loss: 344741.0803006329
Epoch 16, Loss: 664282.75, Validation Loss: 33949

In [ ]:
# Retrieve output weights from the trained model
output_weights = net.output_weight.detach().cpu().numpy().flatten()  # Flatten to make indexing easier

# Define the function for each symbolic transformation
# Assuming funcs are applied cyclically to input features dx, dv, v
input_features = ['dx', 'dv', 'v']
symbolic_equations = []

for i, layer in enumerate(net.hidden_layers):
    func_type = type(layer.function).__name__
    feature = input_features[i % len(input_features)]

    # Depending on the function type, format the symbolic expression
    if func_type == 'Square':
        expr = f"({feature}^2)"
    elif func_type == 'Identity':
        expr = f"({feature})"
    elif func_type == 'Cubic':
        expr = f"({feature}^3)"
    elif func_type == 'Sine':
        expr = f"sin({feature})"
    elif func_type == 'Cosine':
        expr = f"cos({feature})"
    elif func_type == 'Exponential':
        expr = f"exp({feature})"
    elif func_type == 'Logarithm':
        expr = f"log({feature} + 1)"  # Adding 1 inside the log for numerical stability
    else:
        expr = f"({feature})"  # Default case if new types are added later

    # Multiply by the corresponding weight and add to the list
    weight = output_weights[i]
    if weight != 0:  # Only include terms with non-zero weights to simplify the equation
        symbolic_equations.append(f"{weight:.3f} * {expr}")

# Combine all parts into a single equation string
final_equation = " + ".join(symbolic_equations) + " + b"  # Assuming 'b' is the bias term if it exists
print("Symbolic Equation:", final_equation)


Symbolic Equation: -0.754 * (dx^2) + 0.083 * (dv) + -0.074 * (v^3) + 0.223 * sin(dx) + -4.574 * cos(dv) + -0.000 * exp(v) + -1.038 * log(dx + 1) + -0.372 * (dv^2) + -2.015 * (v) + 0.933 * (dx^2) + b


In [ ]:
import torch
import torch.nn as nn

class SymbolicLayer(nn.Module):
    """Simple symbolic layer applying a predefined function to a sliced input."""
    def __init__(self, function, slice_index):
        super().__init__()
        self.function = function
        self.slice_index = slice_index

    def forward(self, x):
        return self.function(x[:, self.slice_index:self.slice_index+1])

class Square(nn.Module):
    def forward(self, x):
        return x ** 2

class Identity(nn.Module):
    def forward(self, x):
        return x

class SymbolicNet(nn.Module):
    """A simple symbolic regression network that ensures a fixed number of outputs."""
    def __init__(self, funcs, input_features):
        super().__init__()
        self.hidden_layers = nn.ModuleList()
        for i in range(5):
            self.hidden_layers.append(SymbolicLayer(funcs[i % len(funcs)], i % input_features))

        # Initialize the output weight using He initialization
        self.output_weight = nn.Parameter(torch.randn(5, 1) * torch.sqrt(torch.tensor(2.0 / 5)))

    def forward(self, x):
        outputs = []
        for layer in self.hidden_layers:
            outputs.append(layer(x))
        x = torch.cat(outputs, dim=1)
        x = torch.matmul(x, self.output_weight)
        return x

# Initialize the network with 5 instances of functions
funcs = [Square(), Identity(), Square(), Identity(), Square()]
net = SymbolicNet(funcs=funcs, input_features=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

# Assume train_loader and validation_loader are defined correctly
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

num_epochs = 100
for epoch in range(num_epochs):
    net.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    scheduler.step()

    net.eval()
    validation_loss = 0
    with torch.no_grad():
        for inputs, targets in validation_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            validation_loss += criterion(outputs, targets).item()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {validation_loss / len(validation_loader)}')


Epoch 1, Loss: 2.7768242359161377, Validation Loss: 3.2524565262130545
Epoch 2, Loss: 1.8897897005081177, Validation Loss: 1.9270253332355354
Epoch 3, Loss: 1.3480901718139648, Validation Loss: 1.288722812374936
Epoch 4, Loss: 0.8935331106185913, Validation Loss: 0.9640559327753284
Epoch 5, Loss: 0.708904504776001, Validation Loss: 0.8772103356409676
Epoch 6, Loss: 0.7686237096786499, Validation Loss: 0.8613172875174994
Epoch 7, Loss: 0.9028732776641846, Validation Loss: 0.8577298635168921
Epoch 8, Loss: 0.7291180491447449, Validation Loss: 0.858019873311248
Epoch 9, Loss: 0.8118047118186951, Validation Loss: 0.8564581033549731
Epoch 10, Loss: 0.8549567461013794, Validation Loss: 0.8569122251076035
Epoch 11, Loss: 0.8033255338668823, Validation Loss: 0.8600225576871559
Epoch 12, Loss: 1.0580894947052002, Validation Loss: 0.8555906875224053
Epoch 13, Loss: 0.8287148475646973, Validation Loss: 0.907615606543384
Epoch 14, Loss: 0.8828138113021851, Validation Loss: 0.8895916108843647
Epoch

In [ ]:
# Retrieve output weights from the trained model
output_weights = net.output_weight.detach().cpu().numpy().flatten()  # Flatten to make indexing easier

# Define the function for each symbolic transformation
# Assuming funcs are applied cyclically to input features dx, dv, v
input_features = ['dx', 'dv', 'v']
symbolic_equations = []

for i, layer in enumerate(net.hidden_layers):
    func_type = type(layer.function).__name__
    feature = input_features[i % len(input_features)]

    # Depending on the function type, format the symbolic expression
    if func_type == 'Square':
        expr = f"({feature}^2)"
    elif func_type == 'Identity':
        expr = f"({feature})"
    else:
        expr = f"({feature})"  # Default case, add more cases if you have more function types

    # Multiply by the corresponding weight and add to the list
    weight = output_weights[i]
    symbolic_equations.append(f"{weight:.3f} * {expr}")

# Combine all parts into a single equation string
final_equation = " + ".join(symbolic_equations) + " + b"
print("Symbolic Equation:", final_equation)


Symbolic Equation: -0.010 * (dx^2) + 0.227 * (dv) + 0.057 * (v^2) + 0.448 * (dx) + -0.038 * (dv^2) + b


In [ ]:
scheduler = torch.optim.lr_scheduler.CyclicLR(
    optimizer,
    base_lr=0.0001,
    max_lr=0.01,
    step_size_up=10,
    mode='exp_range',
    gamma=0.85,
    cycle_momentum=False  # Disable cycle_momentum
)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class InteractionLayer(nn.Module):
    """ Layer to create interaction terms between features. """
    def __init__(self, slice_indices):
        super().__init__()
        self.slice_indices = slice_indices

    def forward(self, x):
        return x[:, self.slice_indices[0]] * x[:, self.slice_indices[1]]

class SymbolicLayer(nn.Module):
    """ Simple symbolic layer applying a predefined function to a sliced input. """
    def __init__(self, function, slice_index):
        super().__init__()
        self.function = function
        self.slice_index = slice_index

    def forward(self, x):
        return self.function(x[:, self.slice_index:self.slice_index+1])

class Square(nn.Module):
    def forward(self, x):
        return x ** 2

class Identity(nn.Module):
    def forward(self, x):
        return x
class InteractionLayer(nn.Module):
    """Layer to create interaction terms between features."""
    def __init__(self, slice_indices):
        super().__init__()
        self.slice_indices = slice_indices

    def forward(self, x):
        return x[:, self.slice_indices[0]:self.slice_indices[0]+1] * x[:, self.slice_indices[1]:self.slice_indices[1]+1]

class SymbolicLayer(nn.Module):
    """Simple symbolic layer applying a predefined function to a sliced input."""
    def __init__(self, function, slice_index):
        super().__init__()
        self.function = function
        self.slice_index = slice_index

    def forward(self, x):
        return self.function(x[:, self.slice_index:self.slice_index+1])
class SymbolicNet(nn.Module):
    def __init__(self, funcs, input_features):
        super().__init__()
        self.hidden_layers = nn.ModuleList()
        for i in range(5):
            self.hidden_layers.append(SymbolicLayer(funcs[i % len(funcs)], i % input_features))
        self.hidden_layers.append(InteractionLayer([0, 1]))  # Interaction between the first and second feature
        self.hidden_layers.append(InteractionLayer([1, 2]))  # Interaction between the second and third feature
        self.output_weight = nn.Parameter(torch.randn(7, 1) * torch.sqrt(torch.tensor(2.0 / 7)))

    def forward(self, x):
        outputs = []
        for layer in self.hidden_layers:
            outputs.append(layer(x))
        x = torch.cat(outputs, dim=1)  # Change dim to 1
        x = torch.matmul(x, self.output_weight)
        return x

funcs = [Square(), Identity(), Square(), Identity(), Square()]
net = SymbolicNet(funcs=funcs, input_features=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(net.parameters(), lr=0.001, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CyclicLR(
    optimizer,
    base_lr=0.0001,
    max_lr=0.01,
    step_size_up=10,
    mode='exp_range',
    gamma=0.85,
    cycle_momentum=False  # Disable cycle_momentum for AdamW
)

num_epochs = 100
for epoch in range(num_epochs):
    net.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    scheduler.step()

    net.eval()
    validation_loss = 0
    with torch.no_grad():
        for inputs, targets in validation_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            validation_loss += criterion(outputs, targets).item()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {validation_loss / len(validation_loader)}')


Epoch 1, Loss: 146.00149536132812, Validation Loss: 127.98545451103887
Epoch 2, Loss: 48.239356994628906, Validation Loss: 38.98533720909795
Epoch 3, Loss: 2.5769479274749756, Validation Loss: 2.742592715009858
Epoch 4, Loss: 1.2530279159545898, Validation Loss: 1.291163305693035
Epoch 5, Loss: 0.8635858297348022, Validation Loss: 0.9402038153213791
Epoch 6, Loss: 0.92360919713974, Validation Loss: 0.8982871853852574
Epoch 7, Loss: 1.012155294418335, Validation Loss: 0.8608705099624924
Epoch 8, Loss: 0.7848567962646484, Validation Loss: 0.8331959564474565
Epoch 9, Loss: 0.8587113618850708, Validation Loss: 0.90227145635629
Epoch 10, Loss: 0.7860592007637024, Validation Loss: 0.8362398449378677
Epoch 11, Loss: 0.7762719988822937, Validation Loss: 0.8636467735978621
Epoch 12, Loss: 0.8336063623428345, Validation Loss: 0.843587412864347
Epoch 13, Loss: 0.7671655416488647, Validation Loss: 0.8407607923580122
Epoch 14, Loss: 0.8173310160636902, Validation Loss: 0.8328217335894138
Epoch 15, 

In [ ]:
# Retrieve output weights from the trained model
output_weights = net.output_weight.detach().cpu().numpy().flatten()  # Flatten to make indexing easier

# Define the function for each symbolic transformation
# Assuming funcs are applied cyclically to input features dx, dv, v
input_features = ['dx', 'dv', 'v']
symbolic_equations = []

for i, layer in enumerate(net.hidden_layers):
    if isinstance(layer, SymbolicLayer):
        func_type = type(layer.function).__name__
        feature = input_features[layer.slice_index]
        # Depending on the function type, format the symbolic expression
        if func_type == 'Square':
            expr = f"({feature}^2)"
        elif func_type == 'Identity':
            expr = f"({feature})"
        else:
            expr = f"({feature})"  # Default case, add more cases if you have more function types
    elif isinstance(layer, InteractionLayer):
        feature1 = input_features[layer.slice_indices[0]]
        feature2 = input_features[layer.slice_indices[1]]
        expr = f"({feature1} * {feature2})"
    else:
        continue  # Skip layers that are not SymbolicLayer or InteractionLayer

    # Multiply by the corresponding weight and add to the list
    weight = output_weights[len(symbolic_equations)]  # Use the current length of symbolic_equations as the index
    symbolic_equations.append(f"{weight:.3f} * {expr}")

# Combine all parts into a single equation string
final_equation = " + ".join(symbolic_equations) + " + b"
print("Symbolic Equation:", final_equation)

Symbolic Equation: -0.011 * (dx^2) + -0.210 * (dv) + 0.057 * (v^2) + 0.450 * (dx) + -0.005 * (dv^2) + -0.002 * (dx * dv) + 0.068 * (dv * v) + b


In [ ]:
"""Methods for regularization to produce sparse networks.

L2 regularization mostly penalizes the weight magnitudes without introducing sparsity.
L1 regularization promotes sparsity.
L1/2 promotes sparsity even more than L1. However, it can be difficult to train due to non-convexity and exploding
gradients close to 0. Thus, we introduce a smoothed L1/2 regularization to remove the exploding gradients."""

import torch
import torch.nn as nn


class L12Smooth(nn.Module):
    def __init__(self):
        super(L12Smooth, self).__init__()

    def forward(self, input_tensor, a=0.05):
        """input: predictions"""
        return l12_smooth(input_tensor, a)


def l12_smooth(input_tensor, a=0.05):
    """Smoothed L1/2 norm"""
    if type(input_tensor) == list:
        return sum([l12_smooth(tensor) for tensor in input_tensor])

    smooth_abs = torch.where(torch.abs(input_tensor) < a,
                             torch.pow(input_tensor, 4) / (-8 * a ** 3) + torch.square(input_tensor) * 3 / 4 / a + 3 * a / 8,
                             torch.abs(input_tensor))

    return torch.sum(torch.sqrt(smooth_abs))

In [ ]:
#regularization

l12_smooth_reg = L12Smooth()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class InteractionLayer(nn.Module):
    """ Layer to create interaction terms between features. """
    def __init__(self, slice_indices):
        super().__init__()
        self.slice_indices = slice_indices

    def forward(self, x):
        return x[:, self.slice_indices[0]] * x[:, self.slice_indices[1]]

class SymbolicLayer(nn.Module):
    """ Simple symbolic layer applying a predefined function to a sliced input. """
    def __init__(self, function, slice_index):
        super().__init__()
        self.function = function
        self.slice_index = slice_index

    def forward(self, x):
        return self.function(x[:, self.slice_index:self.slice_index+1])

class Square(nn.Module):
    def forward(self, x):
        return x ** 2

class Identity(nn.Module):
    def forward(self, x):
        return x
class InteractionLayer(nn.Module):
    """Layer to create interaction terms between features."""
    def __init__(self, slice_indices):
        super().__init__()
        self.slice_indices = slice_indices

    def forward(self, x):
        return x[:, self.slice_indices[0]:self.slice_indices[0]+1] * x[:, self.slice_indices[1]:self.slice_indices[1]+1]

class SymbolicLayer(nn.Module):
    """Simple symbolic layer applying a predefined function to a sliced input."""
    def __init__(self, function, slice_index):
        super().__init__()
        self.function = function
        self.slice_index = slice_index

    def forward(self, x):
        return self.function(x[:, self.slice_index:self.slice_index+1])
class SymbolicNet(nn.Module):
    def __init__(self, funcs, input_features):
        super().__init__()
        self.hidden_layers = nn.ModuleList()
        for i in range(5):
            self.hidden_layers.append(SymbolicLayer(funcs[i % len(funcs)], i % input_features))
        self.hidden_layers.append(InteractionLayer([0, 1]))  # Interaction between the first and second feature
        self.hidden_layers.append(InteractionLayer([1, 2]))  # Interaction between the second and third feature
        self.output_weight = nn.Parameter(torch.randn(7, 1) * torch.sqrt(torch.tensor(2.0 / 7)))

    def forward(self, x):
        outputs = []
        for layer in self.hidden_layers:
            outputs.append(layer(x))
        x = torch.cat(outputs, dim=1)  # Change dim to 1
        x = torch.matmul(x, self.output_weight)
        return x

funcs = [Square(), Identity(), Square(), Identity(), Square()]
net = SymbolicNet(funcs=funcs, input_features=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(net.parameters(), lr=0.001, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CyclicLR(
    optimizer,
    base_lr=0.0001,
    max_lr=0.01,
    step_size_up=10,
    mode='exp_range',
    gamma=0.85,
    cycle_momentum=False  # Disable cycle_momentum for AdamW
)

num_epochs = 100
reg_lambda = 0.01  # Regularization strength

for epoch in range(num_epochs):
    net.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)

        # Calculate the regularization term for each parameter tensor
        reg_term = 0
        for param in net.parameters():
            reg_term += l12_smooth_reg(param)

        # Add the regularization term to the loss
        loss = criterion(outputs, targets) + reg_lambda * reg_term

        loss.backward()
        optimizer.step()
    scheduler.step()

    net.eval()
    validation_loss = 0
    with torch.no_grad():
        for inputs, targets in validation_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            validation_loss += criterion(outputs, targets).item()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {validation_loss / len(validation_loader)}')

Epoch 1, Loss: 65.62969207763672, Validation Loss: 70.86356604853763
Epoch 2, Loss: 2.521970748901367, Validation Loss: 3.1768368829654743
Epoch 3, Loss: 0.715196430683136, Validation Loss: 1.0538983895808836
Epoch 4, Loss: 0.8776630163192749, Validation Loss: 0.8699867008607599
Epoch 5, Loss: 0.8916141986846924, Validation Loss: 0.884906184069718
Epoch 6, Loss: 0.8504906892776489, Validation Loss: 0.8468016137050677
Epoch 7, Loss: 0.8052093386650085, Validation Loss: 0.8370066088966176
Epoch 8, Loss: 1.043770432472229, Validation Loss: 0.8814972095851656
Epoch 9, Loss: 0.8163450360298157, Validation Loss: 0.8643755724158468
Epoch 10, Loss: 1.1290944814682007, Validation Loss: 1.0357853445825698
Epoch 11, Loss: 0.9226245880126953, Validation Loss: 0.8465531853180898
Epoch 12, Loss: 0.8088032603263855, Validation Loss: 0.8345404433298714
Epoch 13, Loss: 0.9490882754325867, Validation Loss: 0.833029832266554
Epoch 14, Loss: 0.8714950084686279, Validation Loss: 0.8376499157917651
Epoch 15

In [ ]:
# Retrieve output weights from the trained model
output_weights = net.output_weight.detach().cpu().numpy().flatten()  # Flatten to make indexing easier

# ... (Symbolic equation retrieval code remains the same)

# Define the function for each symbolic transformation
# Assuming funcs are applied cyclically to input features dx, dv, v
input_features = ['dx', 'dv', 'v']
symbolic_equations = []

for i, layer in enumerate(net.hidden_layers):
    if isinstance(layer, SymbolicLayer):
        func_type = type(layer.function).__name__
        feature = input_features[layer.slice_index]
        # Depending on the function type, format the symbolic expression
        if func_type == 'Square':
            expr = f"({feature}^2)"
        elif func_type == 'Identity':
            expr = f"({feature})"
        else:
            expr = f"({feature})"  # Default case, add more cases if you have more function types
    elif isinstance(layer, InteractionLayer):
        feature1 = input_features[layer.slice_indices[0]]
        feature2 = input_features[layer.slice_indices[1]]
        expr = f"({feature1} * {feature2})"
    else:
        continue  # Skip layers that are not SymbolicLayer or InteractionLayer

    # Multiply by the corresponding weight and add to the list
    weight = output_weights[len(symbolic_equations)]  # Use the current length of symbolic_equations as the index
    symbolic_equations.append(f"{weight:.3f} * {expr}")

# Retrieve the bias term from the model and extract the scalar value
bias = net.output_weight.detach().cpu().numpy()[-1].item()


# Combine all parts into a single equation string
final_equation = " + ".join(symbolic_equations) + f" + {bias:.3f}"
print("Symbolic Equation:", final_equation)

# Calculate the mean squared error on the validation set
net.eval()
val_mse = 0.0
with torch.no_grad():
    for inputs, targets in validation_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = net(inputs)
        val_mse += torch.mean((outputs - targets)**2).item() * inputs.size(0)
val_mse /= len(validation_loader.dataset)

print(f'Validation MSE: {val_mse:.4f}')

In [ ]:
# Retrieve output weights from the trained model
output_weights = net.output_weight.detach().cpu().numpy().flatten()  # Flatten to make indexing easier

# Define the function for each symbolic transformation
# Assuming funcs are applied cyclically to input features dx, dv, v
input_features = ['dx', 'dv', 'v']
symbolic_equations = []

for i, layer in enumerate(net.hidden_layers):
    if isinstance(layer, SymbolicLayer):
        func_type = type(layer.function).__name__
        feature = input_features[layer.slice_index]
        # Depending on the function type, format the symbolic expression
        if func_type == 'Square':
            expr = f"({feature}^2)"
        elif func_type == 'Identity':
            expr = f"({feature})"
        else:
            expr = f"({feature})"  # Default case, add more cases if you have more function types
    elif isinstance(layer, InteractionLayer):
        feature1 = input_features[layer.slice_indices[0]]
        feature2 = input_features[layer.slice_indices[1]]
        expr = f"({feature1} * {feature2})"
    else:
        continue  # Skip layers that are not SymbolicLayer or InteractionLayer

    # Multiply by the corresponding weight and add to the list
    weight = output_weights[len(symbolic_equations)]  # Use the current length of symbolic_equations as the index
    symbolic_equations.append(f"{weight:.3f} * {expr}")

# Retrieve the bias term from the model and extract the scalar value
bias = net.output_weight.detach().cpu().numpy()[-1].item()

# Combine all parts into a single equation string
final_equation = " + ".join(symbolic_equations) + f" + {bias:.3f}"
print("Symbolic Equation:", final_equation)

Symbolic Equation: -0.011 * (dx^2) + -0.127 * (dv) + 0.058 * (v^2) + 0.449 * (dx) + -0.009 * (dv^2) + -0.003 * (dx * dv) + 0.061 * (dv * v) + 0.061


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class InteractionLayer(nn.Module):
    """ Layer to create interaction terms between features. """
    def __init__(self, slice_indices):
        super().__init__()
        self.slice_indices = slice_indices

    def forward(self, x):
        return x[:, self.slice_indices[0]] * x[:, self.slice_indices[1]]

class SymbolicLayer(nn.Module):
    """ Simple symbolic layer applying a predefined function to a sliced input. """
    def __init__(self, function, slice_index):
        super().__init__()
        self.function = function
        self.slice_index = slice_index

    def forward(self, x):
        return self.function(x[:, self.slice_index:self.slice_index+1])

class Square(nn.Module):
    def forward(self, x):
        return x ** 2

class Identity(nn.Module):
    def forward(self, x):
        return x
class InteractionLayer(nn.Module):
    """Layer to create interaction terms between features."""
    def __init__(self, slice_indices):
        super().__init__()
        self.slice_indices = slice_indices

    def forward(self, x):
        return x[:, self.slice_indices[0]:self.slice_indices[0]+1] * x[:, self.slice_indices[1]:self.slice_indices[1]+1]

class SymbolicLayer(nn.Module):
    """Simple symbolic layer applying a predefined function to a sliced input."""
    def __init__(self, function, slice_index):
        super().__init__()
        self.function = function
        self.slice_index = slice_index

    def forward(self, x):
        return self.function(x[:, self.slice_index:self.slice_index+1])
class SymbolicNet(nn.Module):
    def __init__(self, funcs, input_features):
        super().__init__()
        self.hidden_layers = nn.ModuleList()
        for i in range(5):
            self.hidden_layers.append(SymbolicLayer(funcs[i % len(funcs)], i % input_features))
        self.hidden_layers.append(InteractionLayer([0, 1]))  # Interaction between the first and second feature
        self.hidden_layers.append(InteractionLayer([1, 2]))  # Interaction between the second and third feature
        self.output_weight = nn.Parameter(torch.randn(7, 1) * torch.sqrt(torch.tensor(2.0 / 7)))

    def forward(self, x):
        outputs = []
        for layer in self.hidden_layers:
            outputs.append(layer(x))
        x = torch.cat(outputs, dim=1)  # Change dim to 1
        x = torch.matmul(x, self.output_weight)
        return x

funcs = [Square(), Identity(), Square(), Identity(), Square()]
net = SymbolicNet(funcs=funcs, input_features=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(net.parameters(), lr=0.001, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CyclicLR(
    optimizer,
    base_lr=0.0001,
    max_lr=0.01,
    step_size_up=10,
    mode='exp_range',
    gamma=0.85,
    cycle_momentum=False  # Disable cycle_momentum for AdamW
)


# ... (Previous code remains the same)

num_epochs = 100
reg_lambda = 0.01  # Regularization strength

for epoch in range(num_epochs):
    net.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)

        # Calculate the regularization term for each parameter tensor
        reg_term = 0
        for param in net.parameters():
            reg_term += l12_smooth_reg(param)

        # Calculate the mean squared error loss
        loss = torch.mean((outputs - targets)**2) + reg_lambda * reg_term

        train_loss += loss.item() * inputs.size(0)

        loss.backward()
        optimizer.step()
    scheduler.step()

    net.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in validation_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            val_loss += torch.mean((outputs - targets)**2).item() * inputs.size(0)

    train_loss /= len(train_loader.dataset)
    val_loss /= len(validation_loader.dataset)

    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

# Retrieve output weights from the trained model
output_weights = net.output_weight.detach().cpu().numpy().flatten()  # Flatten to make indexing easier

# ... (Symbolic equation retrieval code remains the same)

# Combine all parts into a single equation string
final_equation = " + ".join(symbolic_equations) + f" + {bias:.3f}"
print("Symbolic Equation:", final_equation)

# Calculate the mean squared error on the validation set
net.eval()
val_mse = 0.0
with torch.no_grad():
    for inputs, targets in validation_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = net(inputs)
        val_mse += torch.mean((outputs - targets)**2).item() * inputs.size(0)
val_mse /= len(validation_loader.dataset)

print(f'Validation MSE: {val_mse:.4f}')


Epoch 1, Train Loss: 60008.2837, Validation Loss: 60015.8095
Epoch 2, Train Loss: 35486.3396, Validation Loss: 20525.5486
Epoch 3, Train Loss: 8329.5473, Validation Loss: 2411.6750
Epoch 4, Train Loss: 727.2214, Validation Loss: 159.3366
Epoch 5, Train Loss: 80.1644, Validation Loss: 61.1614
Epoch 6, Train Loss: 50.0448, Validation Loss: 46.2297
Epoch 7, Train Loss: 37.9890, Validation Loss: 36.7087
Epoch 8, Train Loss: 30.2522, Validation Loss: 30.2672
Epoch 9, Train Loss: 25.1868, Validation Loss: 26.0594
Epoch 10, Train Loss: 22.0460, Validation Loss: 22.6565
Epoch 11, Train Loss: 19.7584, Validation Loss: 20.8999
Epoch 12, Train Loss: 18.1975, Validation Loss: 19.1874
Epoch 13, Train Loss: 16.9659, Validation Loss: 17.9404
Epoch 14, Train Loss: 15.9941, Validation Loss: 16.9695
Epoch 15, Train Loss: 15.2516, Validation Loss: 16.3008
Epoch 16, Train Loss: 14.6623, Validation Loss: 15.6810
Epoch 17, Train Loss: 14.1509, Validation Loss: 15.2091
Epoch 18, Train Loss: 13.7614, Validati

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class InteractionLayer(nn.Module):
    """ Layer to create interaction terms between features. """
    def __init__(self, slice_indices):
        super().__init__()
        self.slice_indices = slice_indices

    def forward(self, x):
        return x[:, self.slice_indices[0]] * x[:, self.slice_indices[1]]

class SymbolicLayer(nn.Module):
    """ Simple symbolic layer applying a predefined function to a sliced input. """
    def __init__(self, function, slice_index):
        super().__init__()
        self.function = function
        self.slice_index = slice_index

    def forward(self, x):
        return self.function(x[:, self.slice_index:self.slice_index+1])

class Square(nn.Module):
    def forward(self, x):
        return x ** 2

class Identity(nn.Module):
    def forward(self, x):
        return x
class InteractionLayer(nn.Module):
    """Layer to create interaction terms between features."""
    def __init__(self, slice_indices):
        super().__init__()
        self.slice_indices = slice_indices

    def forward(self, x):
        return x[:, self.slice_indices[0]:self.slice_indices[0]+1] * x[:, self.slice_indices[1]:self.slice_indices[1]+1]

class SymbolicLayer(nn.Module):
    """Simple symbolic layer applying a predefined function to a sliced input."""
    def __init__(self, function, slice_index):
        super().__init__()
        self.function = function
        self.slice_index = slice_index

    def forward(self, x):
        return self.function(x[:, self.slice_index:self.slice_index+1])
class SymbolicNet(nn.Module):
    def __init__(self, funcs, input_features):
        super().__init__()
        self.hidden_layers = nn.ModuleList()
        for i in range(5):
            self.hidden_layers.append(SymbolicLayer(funcs[i % len(funcs)], i % input_features))
        self.hidden_layers.append(InteractionLayer([0, 1]))  # Interaction between the first and second feature
        self.hidden_layers.append(InteractionLayer([1, 2]))  # Interaction between the second and third feature
        self.output_weight = nn.Parameter(torch.randn(7, 1) * torch.sqrt(torch.tensor(2.0 / 7)))

    def forward(self, x):
        outputs = []
        for layer in self.hidden_layers:
            outputs.append(layer(x))
        x = torch.cat(outputs, dim=1)  # Change dim to 1
        x = torch.matmul(x, self.output_weight)
        return x

funcs = [Square(), Identity(), Square(), Identity(), Square()]
net = SymbolicNet(funcs=funcs, input_features=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(net.parameters(), lr=0.001, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CyclicLR(
    optimizer,
    base_lr=0.0001,
    max_lr=0.01,
    step_size_up=10,
    mode='exp_range',
    gamma=0.85,
    cycle_momentum=False  # Disable cycle_momentum for AdamW
)

num_epochs = 100
reg_lambda = 0.01  # Regularization strength

for epoch in range(num_epochs):
    net.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)

        # Calculate the regularization term for each parameter tensor
        reg_term = 0
        for param in net.parameters():
            reg_term += l12_smooth_reg(param)

        # Add the regularization term to the loss
        loss = criterion(outputs, targets) + reg_lambda * reg_term

        loss.backward()
        optimizer.step()
    scheduler.step()

    net.eval()
    validation_loss = 0
    with torch.no_grad():
        for inputs, targets in validation_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            validation_loss += criterion(outputs, targets).item()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {validation_loss / len(validation_loader)}')



Epoch 1, Loss: 63.82646179199219, Validation Loss: 67.69113704826259
Epoch 2, Loss: 4.2512712478637695, Validation Loss: 5.456262558321409
Epoch 3, Loss: 2.156994104385376, Validation Loss: 2.346244288396232
Epoch 4, Loss: 1.6452672481536865, Validation Loss: 1.5758306738696521
Epoch 5, Loss: 0.7156260013580322, Validation Loss: 1.0090347528457642
Epoch 6, Loss: 0.7288315892219543, Validation Loss: 0.8495021462440491
Epoch 7, Loss: 0.7917455434799194, Validation Loss: 0.847599225708201
Epoch 8, Loss: 0.7810760140419006, Validation Loss: 0.8494267750389969
Epoch 9, Loss: 0.9398188591003418, Validation Loss: 0.8616395079636876
Epoch 10, Loss: 0.8598364591598511, Validation Loss: 0.8331296315676049
Epoch 11, Loss: 0.8229178190231323, Validation Loss: 0.89651595716235
Epoch 12, Loss: 0.7914537191390991, Validation Loss: 0.8683391009704976
Epoch 13, Loss: 0.9909659028053284, Validation Loss: 0.9027230045463466
Epoch 14, Loss: 1.0297654867172241, Validation Loss: 0.8383057894586008
Epoch 15,

In [ ]:
# Retrieve output weights from the trained model
output_weights = net.output_weight.detach().cpu().numpy().flatten()  # Flatten to make indexing easier

# ... (Symbolic equation retrieval code remains the same)

# Define the function for each symbolic transformation
# Assuming funcs are applied cyclically to input features dx, dv, v
input_features = ['dx', 'dv', 'v']
symbolic_equations = []

for i, layer in enumerate(net.hidden_layers):
    if isinstance(layer, SymbolicLayer):
        func_type = type(layer.function).__name__
        feature = input_features[layer.slice_index]
        # Depending on the function type, format the symbolic expression
        if func_type == 'Square':
            expr = f"({feature}^2)"
        elif func_type == 'Identity':
            expr = f"({feature})"
        else:
            expr = f"({feature})"  # Default case, add more cases if you have more function types
    elif isinstance(layer, InteractionLayer):
        feature1 = input_features[layer.slice_indices[0]]
        feature2 = input_features[layer.slice_indices[1]]
        expr = f"({feature1} * {feature2})"
    else:
        continue  # Skip layers that are not SymbolicLayer or InteractionLayer

    # Multiply by the corresponding weight and add to the list
    weight = output_weights[len(symbolic_equations)]  # Use the current length of symbolic_equations as the index
    symbolic_equations.append(f"{weight:.3f} * {expr}")

# Retrieve the bias term from the model and extract the scalar value
bias = net.output_weight.detach().cpu().numpy()[-1].item()


# Combine all parts into a single equation string
final_equation = " + ".join(symbolic_equations) + f" + {bias:.3f}"
print("Symbolic Equation:", final_equation)

# Calculate the mean squared error on the validation set
net.eval()
val_mse = 0.0
with torch.no_grad():
    for inputs, targets in validation_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = net(inputs)
        val_mse += torch.mean((outputs - targets)**2).item() * inputs.size(0)
val_mse /= len(validation_loader.dataset)

print(f'Validation MSE: {val_mse:.4f}')

Symbolic Equation: -0.010 * (dx^2) + -0.127 * (dv) + 0.058 * (v^2) + 0.449 * (dx) + -0.009 * (dv^2) + -0.004 * (dx * dv) + 0.060 * (dv * v) + 0.060
Validation MSE: 2.1778


In [ ]:
#making Symbolic Net more complex

import torch
import torch.nn as nn
import torch.optim as optim

class InteractionLayer(nn.Module):
    """Layer to create interaction terms between features."""
    def __init__(self, slice_indices):
        super().__init__()
        self.slice_indices = slice_indices

    def forward(self, x):
        return x[:, self.slice_indices[0]] * x[:, self.slice_indices[1]]

class SymbolicLayer(nn.Module):
    """Simple symbolic layer applying a predefined function to a sliced input."""
    def __init__(self, function, slice_index):
        super().__init__()
        self.function = function
        self.slice_index = slice_index

    def forward(self, x):
        return self.function(x[:, self.slice_index:self.slice_index+1])

class Square(nn.Module):
    def forward(self, x):
        return x ** 2

class Identity(nn.Module):
    def forward(self, x):
        return x

# Updated SymbolicNet class with num_hidden_layers and input_dim parameters
class SymbolicNet(nn.Module):
    def __init__(self, num_hidden_layers, funcs, input_dim=1):
        super().__init__()
        self.hidden_layers = nn.ModuleList()
        for i in range(num_hidden_layers):
            self.hidden_layers.append(SymbolicLayer(funcs[i % len(funcs)], i % input_dim))
        self.hidden_layers.append(InteractionLayer([0, 1]))  # Interaction between the first and second feature
        self.hidden_layers.append(InteractionLayer([1, 2]))  # Interaction between the second and third feature
        self.output_weight = nn.Parameter(torch.randn(num_hidden_layers + 2, 1) * torch.sqrt(torch.tensor(2.0 / (num_hidden_layers + 2))))

    def forward(self, x):
        outputs = []
        for layer in self.hidden_layers:
            output = layer(x)
            if output.dim() == 1:
                output = output.unsqueeze(1)  # Add an extra dimension if the tensor has only one dimension
            outputs.append(output)
        x = torch.cat(outputs, dim=1)
        x = torch.matmul(x, self.output_weight)
        return x

# Add the following lines here
num_hidden_layers = 4  # Specify the desired number of hidden layers
input_dim = 3  # Specify the dimensionality of the input data
funcs = [Square(), Identity(), Square(), Identity(), Square()]
net = SymbolicNet(num_hidden_layers, funcs=funcs, input_dim=input_dim)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(net.parameters(), lr=0.001, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CyclicLR(
    optimizer, base_lr=0.0001, max_lr=0.01, step_size_up=10, mode='exp_range', gamma=0.85, cycle_momentum=False
)

criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(net.parameters(), lr=0.001, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CyclicLR(
    optimizer,
    base_lr=0.0001,
    max_lr=0.01,
    step_size_up=10,
    mode='exp_range',
    gamma=0.85,
    cycle_momentum=False  # Disable cycle_momentum for AdamW
)

num_epochs = 100
reg_lambda = 0.01  # Regularization strength

for epoch in range(num_epochs):
    net.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)

        # Calculate the regularization term for each parameter tensor
        reg_term = 0
        for param in net.parameters():
            reg_term += l12_smooth_reg(param)

        # Add the regularization term to the loss
        loss = criterion(outputs, targets) + reg_lambda * reg_term

        loss.backward()
        optimizer.step()
    scheduler.step()

    net.eval()
    validation_loss = 0
    with torch.no_grad():
        for inputs, targets in validation_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            validation_loss += criterion(outputs, targets).item()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {validation_loss / len(validation_loader)}')



Epoch 1, Loss: 61.1967658996582, Validation Loss: 67.471561625034
Epoch 2, Loss: 27.220211029052734, Validation Loss: 32.745859894571424
Epoch 3, Loss: 6.92686128616333, Validation Loss: 7.726172012618825
Epoch 4, Loss: 1.65268874168396, Validation Loss: 2.121576909777484
Epoch 5, Loss: 0.8786890506744385, Validation Loss: 0.9822866750668876
Epoch 6, Loss: 0.9550784230232239, Validation Loss: 0.9087459656256663
Epoch 7, Loss: 0.8265520930290222, Validation Loss: 0.8580439392524429
Epoch 8, Loss: 0.9738342761993408, Validation Loss: 0.8357940542547009
Epoch 9, Loss: 0.6605714559555054, Validation Loss: 0.8357748894751826
Epoch 10, Loss: 0.9118329286575317, Validation Loss: 0.8580577818653251
Epoch 11, Loss: 0.6513704657554626, Validation Loss: 0.938098128083386
Epoch 12, Loss: 0.7552004456520081, Validation Loss: 0.8412725578380537
Epoch 13, Loss: 1.1613324880599976, Validation Loss: 0.8432387143750734
Epoch 14, Loss: 0.896597683429718, Validation Loss: 0.8370290526860877
Epoch 15, Loss

In [ ]:
# Retrieve output weights from the trained model
output_weights = net.output_weight.detach().cpu().numpy().flatten()  # Flatten to make indexing easier

# ... (Symbolic equation retrieval code remains the same)

# Define the function for each symbolic transformation
# Assuming funcs are applied cyclically to input features dx, dv, v
input_features = ['dx', 'dv', 'v']
symbolic_equations = []

for i, layer in enumerate(net.hidden_layers):
    if isinstance(layer, SymbolicLayer):
        func_type = type(layer.function).__name__
        feature = input_features[layer.slice_index]
        # Depending on the function type, format the symbolic expression
        if func_type == 'Square':
            expr = f"({feature}^2)"
        elif func_type == 'Identity':
            expr = f"({feature})"
        else:
            expr = f"({feature})"  # Default case, add more cases if you have more function types
    elif isinstance(layer, InteractionLayer):
        feature1 = input_features[layer.slice_indices[0]]
        feature2 = input_features[layer.slice_indices[1]]
        expr = f"({feature1} * {feature2})"
    else:
        continue  # Skip layers that are not SymbolicLayer or InteractionLayer

    # Multiply by the corresponding weight and add to the list
    weight = output_weights[len(symbolic_equations)]  # Use the current length of symbolic_equations as the index
    symbolic_equations.append(f"{weight:.3f} * {expr}")

# Retrieve the bias term from the model and extract the scalar value
bias = net.output_weight.detach().cpu().numpy()[-1].item()


# Combine all parts into a single equation string
final_equation = " + ".join(symbolic_equations) + f" + {bias:.3f}"
print("Symbolic Equation:", final_equation)

# Calculate the mean squared error on the validation set
net.eval()
val_mse = 0.0
with torch.no_grad():
    for inputs, targets in validation_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = net(inputs)
        val_mse += torch.mean((outputs - targets)**2).item() * inputs.size(0)
val_mse /= len(validation_loader.dataset)

print(f'Validation MSE: {val_mse:.4f}')

Symbolic Equation: -0.011 * (dx^2) + -0.142 * (dv) + 0.058 * (v^2) + 0.447 * (dx) + -0.004 * (dx * dv) + 0.066 * (dv * v) + 0.066
Validation MSE: 2.2763
